# <center>Regressão - Exercício</center>
___

<a id="exercicio"></a>
## 6. Exercícios

Esse notebook contém os exercícios da aula de regressão. Siga as instruções para aplicar o que foi aprendido nessa aula para outra base de dados para tentar prever os preços de carros. Essa base de dados foi disponibilizada junto com o notebook.

Vamos começasr lendo os dados.

In [ ]:
import pandas as pd
import numpy as np
import math
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

In [ ]:
cars_df = pd.read_csv("./data/OLX_Car_Data.csv", encoding = 'unicode_escape')

cars_df.describe(include='all')

Utilizando o método ```describe``` conseguimos ver algumas informações necessárias para fazer uma limpeza básica nos nossos dados. Veja abaixo.

In [ ]:
# renomeando as colunas para remover os espaços
cars_df.rename(columns = {'KMs Driven':'KMs_Driven', 
                          'Registered City':'Registered_City', 
                          'Transaction Type':'Transaction_Type'},
               inplace = True)

# Já que a coluna Model tem muitas categorias, vamos nomear todas com menos de 400 observações de 'Outros'
models = ['Alto', 'Corolla GLI', 'Mehran VX', 'Mehran VXR', 'Vitz', 'Bolan', 'Cuore', 'Corolla XLI', 'Mira']
cars_df.loc[~cars_df['Model'].isin(models),'Model'] = 'Outros'

# removendo valores nulos
cars_df.dropna(axis=0, inplace=True)

# removendo outliers
low = 0.05
high = 0.95
quant_df = cars_df.quantile([low, high])
cars_df = cars_df[(cars_df.KMs_Driven < quant_df.loc[high, "KMs_Driven"]) & 
                  (cars_df.KMs_Driven > quant_df.loc[low, "KMs_Driven"]) &
                  (cars_df.Price < quant_df.loc[high, "Price"]) & 
                  (cars_df.Price > quant_df.loc[low, "Price"]) &
                  (cars_df.Year < quant_df.loc[high, "Year"]) & 
                  (cars_df.Year > quant_df.loc[low, "Year"])]

# transformando as variáveis binárias
cars_df['is_new'] = 1
cars_df.loc[cars_df.Condition == 'Used', 'is_new'] = 0
cars_df.drop(['Condition'], axis=1, inplace=True)

cars_df['is_cash'] = 0
cars_df.loc[cars_df.Transaction_Type=='Cash', 'is_cash'] = 1
cars_df.drop('Transaction_Type', axis=1, inplace=True)

# aplicando o One Hot Encoding
cars_df = pd.concat([cars_df.drop("Brand", axis=1), pd.get_dummies(cars_df.Brand, prefix='Brand')], axis=1)
cars_df = pd.concat([cars_df.drop("Fuel", axis=1), pd.get_dummies(cars_df.Fuel, prefix='Fuel')], axis=1)
cars_df = pd.concat([cars_df.drop("Registered_City", axis=1), pd.get_dummies(cars_df.Registered_City, prefix='RCity')], axis=1)
cars_df = pd.concat([cars_df.drop("Model", axis=1), pd.get_dummies(cars_df.Model, prefix='Model')], axis=1)

cars_df.describe(include='all')

In [ ]:
y, X = cars_df.Price, cars_df.drop('Price', axis=1)

print("X", X.shape)
print("y", y.shape)

In [ ]:
seed = 1

# separando treino e teste
X_training, X_test, y_training, y_test = train_test_split(X, y, random_state=seed, test_size=0.25) #, stratify=y)
print("Test set X", X_test.shape)
print("Test set y", y_test.shape)

# separando treino e validação
X_train, X_val, y_train, y_val = train_test_split(X_training, y_training, random_state=seed,
                                                  test_size=0.33) #, stratify=y_training)

print("Train set X", X_train.shape)
print("Train set y", y_train.shape)
print("Validation set X", X_val.shape)
print("Validation set y", y_val.shape)

#### Exercício 1

**Substitua as lacunas** para treinar um modelo de *random forest* com validação cruzada e *grid search*. O *grid search* deve variar, pelo menos, os parâmetros *n_estimators* e *max_depth*, mas você pode incluir outros. Após o treino, crie o *dataframe* ```cv_results``` com os resultados de cada iteração e o dicionário ```cv_best_params``` com os valores da melhor combinação de parâmetros.

In [ ]:
# Random Forest com validação cruzada e Grid Search

# definindo os valores possíveis dos parâmetros a serem testados
params = {}

# criando o objeto do modelo com RandomForestRegressor
rf_model_cv_gs = ________________()

# criando o objeto do grid search com GridSearchCV
grid_search = ___________(rf_model_cv_gs, param_grid=params, return_train_score=True, scoring='neg_root_mean_squared_error')

# treinando o modelo com o grid search
grid_search.________(X_training, y_training)

# pegando os resultados da validação cruzada (cv_results)
cv_results = pd.DataFrame(grid_search.________)

# pegando e imprimindo a melhor combinação de hiperparâmetros
cv_best_params = grid_search.________
print('\n Best hyperparameters:')
print(cv_best_params)

#### Exercício 2

**Substitua as lacunas** para treinar o modelo final de *random forest* com os melhores parâmetros obtidos no *grid search* do exercício anterior.

In [ ]:
# imprimindo o score médio nas bases de treino
print("Average accuracy on train set: {:.3f} +/- {:.3f}".format(cv_results[cv_results.rank_test_score == 1].mean_train_score.values[0],
                                                                     cv_results[cv_results.rank_test_score == 1].std_train_score.values[0])) 
# imprimindo o score médio nas bases de validação
print("Average accuracy on validation set: {:.3f} +/- {:.3f}".format(cv_results[cv_results.rank_test_score == 1].mean_test_score.values[0],
                                                                     cv_results[cv_results.rank_test_score == 1].std_test_score.values[0])) 

# configurando o modelo com a melhor combinação de hiperparâmetros
rf_model_cv_gs.________(n_estimators = cv_best_params['n_estimators'],
                        max_depth = cv_best_params['max_depth'],
                        ______)

# treinando um modelo com a melhor combinação de hiperparâmetros
rf_model_cv_gs.________(X_training, y_training)
best_model_params = rf_model_cv_gs.get_params()

#### Exercício 3

**Substitua as lacunas** para obter um *dataframe* com a importância de cada variável do modelo de *random forest*.

In [ ]:
# desenhando o gráfico de impoartância de variáveis
features = X_training.______
importances = rf_model_cv_gs.______
indices = np.argsort(importances)

feature_importances_df = pd.DataFrame({'features': features,
                                       'importances': importances})

plt.title('Feature Importances')
plt.barh(range(len(importances[indices][-15:])), importances[indices][-15:], color='b', align='center')
plt.yticks(range(len(importances[indices][-15:])), [features[i] for i in indices[-15:]])
plt.xlabel('Relative Importance')
plt.show()

#### Exercício 4

**Substitua as lacunas** para calcular o RMSE do modelo de *random forest* final na base de teste.

In [ ]:
y_test_pred_rf = rf_model_cv_gs.______(X_test)
rmse_test_rf = math.sqrt(mean_squared_error(y_test, ______))
print(rmse_test_rf)

#### Exercício 5

**Substitua as lacunas** para treinar um modelo de *Light GBM* com validação cruzada e *grid search*. O *grid search* deve variar, pelo menos, os parâmetros *learning_rate* e *n_estimators*, mas você pode incluir outros. Após o treino, crie o *dataframe* ```cv_results``` com os resultados de cada iteração e o dicionário ```cv_best_params``` com os valores da melhor combinação de parâmetros.

In [ ]:
# Light GBM com validação cruzada e Grid Search

# definindo os valores possíveis dos parâmetros a serem testados
# params = {'max_depth': [10, 50, 100],
#           'learning_rate': [0.01, 0.03, 0.1, 0.5],
#           'num_iterations': [100, 200, 500],
#           'min_data_in_leaf': [20, 50],
#           'min_gain_to_split': [0., 1, 5]}
params = {____}

# criando o objeto do modelo com XGBClassifier
lgb_model_cv_gs = lgb.__________()

# criando o objeto do grid search com GridSearchCV
grid_search = _________(lgb_model_cv_gs, param_grid=params, return_train_score=True, scoring='neg_root_mean_squared_error')

# treinando o modelo com o grid search
grid_search.______(X_training, y_training)

# pegando os resultados da validação cruzada (cv_results)
cv_results = pd.DataFrame(grid_search._________)

# pegando e imprimindo a melhor combinação de hiperparâmetros
cv_best_params = grid_search._________
print('\n Best hyperparameters:')
print(cv_best_params)

#### Exercício 6

**Substitua as lacunas** para treinar o modelo final de *light GBM* com os melhores parâmetros obtidos no *grid search* do exercício anterior.

In [ ]:
# imprimindo o score médio nas bases de treino
print("Average accuracy on train set: {:.3f} +/- {:.3f}".format(cv_results[cv_results.rank_test_score == 1].mean_train_score.values[0],
                                                                     cv_results[cv_results.rank_test_score == 1].std_train_score.values[0]))

# imprimindo o score médio nas bases de validação
print("Average accuracy on validation set: {:.3f} +/- {:.3f}".format(cv_results[cv_results.rank_test_score == 1].mean_test_score.values[0],
                                                                     cv_results[cv_results.rank_test_score == 1].std_test_score.values[0]))

# configurando o modelo com a melhor combinação de hiperparâmetros
lgb_model_cv_gs.__________(learning_rate = cv_best_params['learning_rate'],
                           n_estimators = cv_best_params['n_estimators'],
                           ______)

# treinando um modelo com a melhor combinação de hiperparâmetros
lgb_model_cv_gs._____(X_training, y_training)
best_model_params = lgb_model_cv_gs.get_params()

#### Exercício 7

**Substitua as lacunas** para obter um *dataframe* com a importância de cada variável do modelo de *light GBM*.

In [ ]:
# desenhando o gráfico de impoartância de variáveis
features = X_training.______
importances = lgb_model_cv_gs.______
indices = np.argsort(importances)

feature_importances_df = pd.DataFrame({'features': features,
                                       'importances': importances})

plt.title('Feature Importances')
plt.barh(range(len(importances[indices][-15:])), importances[indices][-15:], color='b', align='center')
plt.yticks(range(len(importances[indices][-15:])), [features[i] for i in indices[-15:]])
plt.xlabel('Relative Importance')
plt.show()

#### Exercício 8

**Substitua as lacunas** para calcular o RMSE do modelo de *light GBM* final na base de teste.

In [ ]:
y_test_pred_lgb = lgb_model_cv_gs.______(X_test)
rmse_test_lgb = math.sqrt(mean_squared_error(y_test, ______))
print(rmse_test_lgb)

Os resultados dos modelos criados até aqui fazem sentido para você?

Vamos desenhar gráficos para comparar as duas previsões como fizamos com os dados de **boston**, e usá-los para responder as demais perguntas:

In [ ]:
# Desenhando o gráfico de valores previstos por valores reais para ambos os modelos

plt.figure(figsize=(16,10))
plt.title('Pakistan car prices - Predicted vs Real',fontsize=20)
df = pd.DataFrame({'real':y_val,'Random Forest':rf_model_cv_gs.predict(X_val),
                   'LGBM':lgb_model_cv_gs.predict(X_val)})
df = df.sample(200)
df.sort_values(by=['real'],ascending=True,inplace=True)
df = df.reset_index(drop=True)
plt.plot(df)
plt.legend(['Real price','Predicted RF price','Predicted LGBM price'],fontsize=20)
plt.ylabel('$ Price',fontsize=20)
plt.xlabel('Observations ordered by price',fontsize=20)
plt.show()

plt.figure(figsize=(16,10))
plt.title('Pakistan car prices - Predicted vs Real',fontsize=20)
plt.scatter(x=df['real'],y=df['Random Forest'],c='y')
plt.scatter(x=df['real'],y=df['LGBM'],c='g')
plt.plot([0,1750000],[0,1750000],'k--')
# plt.axis([0,1750000,0,1750000])
plt.legend(['Real price','Predicted RF price','Predicted LGBM price'],fontsize=20)
plt.xlabel('Real price',fontsize=20)
plt.ylabel('Predicted price',fontsize=20)
plt.show()

#### Exercício 9
*(Resposta aberta, não contabilizara pontos, o objetivo é refletir sobre a questão/tópico)*

Qual modelo parece ter menos *over-fitting*? Explique

In [ ]:
# suas respostas


#### Exercício 10
*(Resposta aberta, não contabilizara pontos, o objetivo é refletir sobre a questão/tópico)*

Qual modelo faz a melhor previsão dos preços dos carros em cada faixa de preço?

In [ ]:
# suas respostas


#### Exercício 11
*(Resposta aberta, não contabilizara pontos, o objetivo é refletir sobre a questão/tópico)*

Qual modelo vocês utilizaria afinal? Explique

In [ ]:
# suas respostas
